<a href="https://colab.research.google.com/github/vanTHkrab/COE64-335-Project-2025/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This is a sample Jupyter Notebook

Below is an example of a code cell.
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, f1_score, roc_auc_score
import joblib



In [ ]:
df = pd.read_csv("data/raw-rain-data.csv")

In [ ]:
df.head(5)

Cleaning

1. Check Missing value

In [ ]:

print(df.describe(include="all"))

2.ตัด Column PROV_ID

In [ ]:
df = df.drop(columns=["PROV_T"])


In [ ]:
df.head(5)

3.สร้าง Label >= 90 คือตก

In [ ]:
df["Rain"] = (df["AvgRain"] >= 90).astype(int)

In [ ]:
df.head(5)

Feature Engineering

1.Add coulumn

- Seasonality

In [ ]:
df["month_sin"] = np.sin(2*np.pi*df["MONTH"]/12)
df["month_cos"] = np.cos(2*np.pi*df["MONTH"]/12)

In [ ]:
df.head(5)

In [ ]:
df = df.drop(columns=["MONTH"])

In [ ]:
df.head(3)

# Train/Test Split

In [ ]:
X = df[["PROV_ID", "month_sin", "month_cos"]]
y = df["Rain"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
print("X_train sample:")
print(X_train.head())

print("\ny_train sample:")
print(y_train.head())

print("\nX_test sample:")
print(X_test.head())

print("\ny_test sample:")
print(y_test.head())


********** Traning ************
and scaling

# Logistic Regression

In [ ]:
# log_pipe = Pipeline([
#     ("scaler", StandardScaler()),
#     ("clf", LogisticRegression(max_iter=1000))
# ])
# log_pipe.fit(X_train, y_train)

In [ ]:
# y_pred = log_pipe.predict(X_test)
# y_proba = log_pipe.predict_proba(X_test)[:, 1]   # ใช้ proba ของ Rain=1
#
# print("=== Logistic Regression ===")
# print(classification_report(y_test, y_pred, digits=4))
# print(confusion_matrix(y_test, y_pred))
#
# # เพิ่ม F1 และ ROC-AUC
# f1 = f1_score(y_test, y_pred)
# auc = roc_auc_score(y_test, y_proba)
# print(f"F1-score (test): {f1:.4f}")
# print(f"ROC-AUC   (test): {auc:.4f}")

# Decision Tree

In [ ]:
# tree_pipe = Pipeline([
#     ("scaler", StandardScaler()),   # จริง ๆ ไม่จำเป็นกับ tree
#     ("clf", DecisionTreeClassifier(random_state=42))
# ])
# tree_pipe.fit(X_train, y_train)

In [ ]:
# y_pred = tree_pipe.predict(X_test)
# print("=== Decision Tree ===")
# print(classification_report(y_test, y_pred, digits=4))
# print(confusion_matrix(y_test, y_pred))
# # เพิ่ม F1 และ ROC-AUC
# f1 = f1_score(y_test, y_pred)
# auc = roc_auc_score(y_test, y_proba)
# print(f"F1-score (test): {f1:.4f}")
# print(f"ROC-AUC   (test): {auc:.4f}")

# Random Forest

In [ ]:
# rf_pipe = Pipeline([
#     ("scaler", StandardScaler()),   # ไม่จำเป็นกับ forest เช่นกัน
#     ("clf", RandomForestClassifier(random_state=42))
# ])
# rf_pipe.fit(X_train, y_train)

In [ ]:
# y_pred = rf_pipe.predict(X_test)
# print("=== Random Forest ===")
# print(classification_report(y_test, y_pred, digits=4))
# print(confusion_matrix(y_test, y_pred))
# # เพิ่ม F1 และ ROC-AUC
# f1 = f1_score(y_test, y_pred)
# auc = roc_auc_score(y_test, y_proba)
# print(f"F1-score (test): {f1:.4f}")
# print(f"ROC-AUC   (test): {auc:.4f}")

# Hyperparameter Tunning

## Grid Search

In [ ]:
from typing import Dict, Any, Optional, List, Tuple
import numpy as np
import pandas as pd

from sklearn.model_selection import GridSearchCV, StratifiedKFold, StratifiedShuffleSplit
from sklearn.metrics import (
    f1_score, roc_auc_score, average_precision_score,
    classification_report, confusion_matrix, make_scorer,
    balanced_accuracy_score
)

# ใช้ร่วมกับ:
# cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# scoring = "f1"   # (ยังใช้ได้) แต่ในฟังก์ชันนี้จะสร้าง scorers หลายตัวให้อัตโนมัติ

def _get_proba_or_decision(estimator, X) -> np.ndarray:
    """คืนค่า 'ความน่าจะเป็นของ positive class' (binary) หรือ 'matrix proba' (multiclass)
       ถ้าไม่มี predict_proba จะ fallback เป็น decision_function และสเกลให้อยู่ [0,1] แบบ min-max"""
    if hasattr(estimator, "predict_proba"):
        proba = estimator.predict_proba(X)
        return proba
    elif hasattr(estimator, "decision_function"):
        dec = estimator.decision_function(X)
        # binary: (n,), multiclass: (n, n_classes)
        if dec.ndim == 1:
            dec = dec.reshape(-1, 1)
        # min-max per column
        dmin = dec.min(axis=0, keepdims=True)
        dmax = dec.max(axis=0, keepdims=True)
        rng = np.clip(dmax - dmin, 1e-12, None)
        p = (dec - dmin) / rng
        # ถ้า binary จะได้คอลัมน์เดียว ให้รวมเป็น 2 คอลัมน์ [1-p, p] เพื่อความสอดคล้อง
        if p.shape[1] == 1:
            p = np.hstack([1 - p, p])
        return p
    else:
        # ไม่มีทั้งสองอย่าง → ใช้ predict แล้ว cast เป็น proba (จำเป็นจริง ๆ เท่านั้น)
        pred = estimator.predict(X)
        # one-hot dummy
        classes = np.unique(pred)
        n_classes = classes.size
        out = np.zeros((pred.shape[0], n_classes))
        for i, c in enumerate(classes):
            out[:, i] = (pred == c).astype(float)
        # ถ้าเป็น binary แปลงให้คอลัมน์ 1 คือ positive (assume class 1 ถ้ามี)
        if n_classes == 1:
            out = np.hstack([1 - out, out])
        return out

def _best_threshold_by_f1(y_true: np.ndarray, pos_score: np.ndarray) -> Tuple[float, float]:
    """สแกน threshold 200 จุด หา threshold ที่ให้ F1 สูงสุด (binary)"""
    thresh_grid = np.linspace(0.0, 1.0, 201)
    best_f1, best_t = -1.0, 0.5
    for t in thresh_grid:
        y_hat = (pos_score >= t).astype(int)
        f1 = f1_score(y_true, y_hat, zero_division=0)
        if f1 > best_f1:
            best_f1, best_t = f1, t
    return best_t, best_f1

def run_grid_search(
    pipe,
    grid,
    *,
    cv=None,
    primary_metric: str = "f1",      # metric ที่จะใช้ refit
    tune_threshold: bool = True,     # ถ้าเป็น binary จะจูน threshold บน val split
    show_top_k: int = 10,            # แสดง top-k แถวแรกของ cv_results_
    random_state: int = 42,
):
    # ===== 1) CV & scorers =====
    if cv is None:
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)

    # สร้าง scorers หลายตัวเพื่อดูภาพรวม
    scorers: Dict[str, Any] = {
        "f1": make_scorer(f1_score),
        "f1_macro": "f1_macro",
        "f1_weighted": "f1_weighted",
        "balanced_acc": make_scorer(balanced_accuracy_score),
        # หมายเหตุ: roc_auc จะใช้ได้ดีใน binary; ถ้าเป็น multiclass ใช้ roc_auc_ovr/ovo
        "roc_auc": "roc_auc",
        "roc_auc_ovr": "roc_auc_ovr",
        "ap": "average_precision",  # PR-AUC (binary)
    }

    # ===== 2) GridSearchCV (error_score="nan" จะข้ามคอมโบที่ไม่เข้ากัน) =====
    search = GridSearchCV(
        estimator=pipe,
        param_grid=grid,
        scoring=scorers,
        refit=primary_metric,
        cv=cv,
        n_jobs=-1,
        verbose=2,
        error_score=np.nan,
        return_train_score=False,
    )
    search.fit(X_train, y_train)  # อาศัยตัวแปร global เหมือนโค้ดเดิมของคุณ

    best_model = search.best_estimator_
    print("\n[Best params]")
    print(search.best_params_)
    print(f"[Best CV {primary_metric}]: {search.best_score_:.6f}")

    # ===== 3) แสดงผล CV top-k =====
    cv_df = pd.DataFrame(search.cv_results_)
    rank_col = f"rank_test_{primary_metric}"
    score_col = f"mean_test_{primary_metric}"
    cols = [rank_col, score_col] + [c for c in cv_df.columns if c.startswith("param_")]
    cv_view = cv_df.sort_values(rank_col)[cols].head(show_top_k)
    print(f"\n[Top {show_top_k} by {primary_metric}]")
    print(cv_view.to_string(index=False))

    # ===== 4) ประเมินบน test set + (ออปชัน) ปรับ threshold สำหรับ binary =====
    y_pred_default = best_model.predict(X_test)

    # เตรียมคะแนน/โปรบาบิลิตี
    proba_test = _get_proba_or_decision(best_model, X_test)
    n_classes = proba_test.shape[1]
    is_binary = (n_classes == 2)

    if is_binary:
        pos_proba_test = proba_test[:, 1]

        # default (threshold = 0.5)
        f1_default = f1_score(y_test, y_pred_default)
        auc_roc = roc_auc_score(y_test, pos_proba_test)
        ap = average_precision_score(y_test, pos_proba_test)

        if tune_threshold:
            # split train → (sub-train, val) เพื่อหา threshold ที่ให้ F1 ดีสุด
            sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=random_state)
            (tr_idx, val_idx), = sss.split(X_train, y_train)
            # ฟิตใหม่บน sub-train เพื่อป้องกันข้อมูลรั่ว (หรือจะใช้ best_model เดิมบน X_train ก็ได้ แต่เข้มงวดหน่อย)
            best_model.fit(X_train[tr_idx], y_train[tr_idx])
            proba_val = _get_proba_or_decision(best_model, X_train[val_idx])[:, 1]
            t_star, val_f1 = _best_threshold_by_f1(y_train[val_idx], proba_val)

            # ประเมินบน test ด้วย threshold ใหม่
            proba_test_star = _get_proba_or_decision(best_model, X_test)[:, 1]
            y_pred_star = (proba_test_star >= t_star).astype(int)
            f1_star = f1_score(y_test, y_pred_star)

            print("\n[Test metrics (binary)]")
            print(f" F1 (default 0.5): {f1_default:.4f}")
            print(f" F1 (tuned @ {t_star:.3f}): {f1_star:.4f}  [val-F1={val_f1:.4f}]")
            print(f" ROC-AUC: {auc_roc:.4f}")
            print(f" PR-AUC (AP): {ap:.4f}")

            # สรุปรีพอร์ต/คอนฟิวชันแมทริกซ์ด้วย threshold ที่ดีกว่า
            y_pred = y_pred_star if f1_star >= f1_default else y_pred_default
        else:
            print("\n[Test metrics (binary)]")
            print(f" F1: {f1_default:.4f}")
            print(f" ROC-AUC: {auc_roc:.4f}")
            print(f" PR-AUC (AP): {ap:.4f}")
            y_pred = y_pred_default

    else:
        # multiclass
        y_pred = y_pred_default
        print("\n[Test metrics (multiclass)]")
        print(f" F1-macro: {f1_score(y_test, y_pred, average='macro'):.4f}")
        print(f" F1-weighted: {f1_score(y_test, y_pred, average='weighted'):.4f}")
        # ovr AUC (ต้องการ proba/score ทุกรุ่น)
        try:
            auc_ovr = roc_auc_score(y_test, proba_test, multi_class="ovr")
            print(f" ROC-AUC (OvR): {auc_ovr:.4f}")
        except Exception:
            pass

    # ===== 5) รายงานละเอียด =====
    print("\n[Classification report]")
    print(classification_report(y_test, y_pred, digits=4))
    print("[Confusion matrix]")
    print(confusion_matrix(y_test, y_pred))

    # ส่งคืนทั้งโมเดลและตารางผล CV เผื่อคุณอยากเก็บลงไฟล์/วิเคราะห์ต่อ
    return best_model, cv_df


### - Logistic Regession

In [ ]:
lr_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=4000, multi_class="auto"))
])

In [ ]:
# grid_lr = [
#     {
#         "clf__solver": ["lbfgs", "newton-cg", "liblinear"],
#         "clf__penalty": ["l2", "none"] if "liblinear" not in ["liblinear"] else ["l2"],  # อธิบายด้านล่าง
#         "clf__C": [0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0, 100.0],
#         "clf__class_weight": [None, "balanced"],
#         "clf__fit_intercept": [True, False],
#         "clf__tol": [1e-4, 1e-5],
#     },
#     {
#         "clf__solver": ["liblinear"],
#         "clf__penalty": ["l1", "l2"],   # liblinear รองรับ L1 ด้วย
#         "clf__C": [0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0, 100.0],
#         "clf__class_weight": [None, "balanced"],
#         "clf__fit_intercept": [True, False],
#         "clf__tol": [1e-4, 1e-5],
#     },
#     {
#         "clf__solver": ["saga"],
#         "clf__penalty": ["l1", "l2", "elasticnet"],
#         "clf__l1_ratio": [0.1, 0.5, 0.9],  # ใช้เมื่อ penalty="elasticnet"
#         "clf__C": [0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0, 100.0],
#         "clf__class_weight": [None, "balanced"],
#         "clf__fit_intercept": [True, False],
#         "clf__tol": [1e-4, 1e-5],
#     },
# ]
#
# best_lr = run_grid_search(lr_pipe, grid_lr)

In [ ]:
# lr_pipe = Pipeline([
#     ("scaler", StandardScaler()),
#     ("clf", LogisticRegression(max_iter=4000, solver="lbfgs"))
# ])
#
# grid_lr = {
#     "clf__C": [0.1, 0.5, 1.0, 2.0, 10.0],   # คุมความแรง regularization
#     "clf__class_weight": [None, "balanced"] # ถ่วง class ถ้า skew
# }
#
#
# best_lr = run_grid_search(lr_pipe, grid_lr)


In [ ]:
# ใช้กับ Pipeline ที่ตั้งชื่อ estimator ว่า 'clf' (เช่น lr_pipe = Pipeline([... , ('clf', LogisticRegression())]))

grid_lr_long = [
    # ===== lbfgs / newton-cg / sag: รองรับเฉพาะ L2 และ none =====
    {
        "clf__solver": ["lbfgs", "newton-cg", "sag"],
        "clf__penalty": ["l2", "none"],
        "clf__C": [1e-5, 3e-5, 1e-4, 3e-4, 1e-3, 3e-3, 1e-2, 0.05, 0.1, 0.2, 0.5,
                   1, 2, 5, 10, 20, 50, 100, 200, 500, 1_000, 5_000],
        "clf__tol": [1e-2, 3e-3, 1e-3, 3e-4, 1e-4, 3e-5, 1e-5, 1e-6],
        "clf__max_iter": [100, 200, 500, 1_000, 2_000, 5_000],
        "clf__fit_intercept": [True, False],
        "clf__class_weight": [None, "balanced"],
        "clf__multi_class": ["auto", "ovr", "multinomial"],
        # sag มักไวกับ learning conditions; ให้ลองทั้งสามตัวรวมกันเพื่อเปรียบเทียบ
        "clf__warm_start": [False, True],
    },

    # ===== liblinear: รองรับ L1/L2, ใช้ ovr เท่านั้น; dual ใช้กับ L2 เท่านั้น =====
    {
        "clf__solver": ["liblinear"],
        "clf__penalty": ["l1", "l2"],
        "clf__dual": [False, True],          # จะถูกใช้เฉพาะคู่กับ penalty='l2'
        "clf__intercept_scaling": [0.1, 0.5, 1.0, 2.0, 5.0],
        "clf__C": [1e-5, 1e-4, 1e-3, 1e-2, 0.05, 0.1, 0.2, 0.5,
                   1, 2, 5, 10, 20, 50, 100, 200, 500, 1_000, 2_000],
        "clf__tol": [1e-2, 1e-3, 1e-4, 1e-5, 1e-6],
        "clf__max_iter": [100, 200, 500, 1_000, 2_000],
        "clf__fit_intercept": [True, False],
        "clf__class_weight": [None, "balanced"],
        "clf__multi_class": ["ovr"],
        "clf__warm_start": [False, True],
    },

    # ===== saga: รองรับ L1/L2/elasticnet, รองรับ multinomial =====
    {
        "clf__solver": ["saga"],
        "clf__penalty": ["l1", "l2", "elasticnet"],
        "clf__l1_ratio": [0.0, 0.1, 0.25, 0.4, 0.5, 0.6, 0.75, 0.9, 1.0],  # ใช้เมื่อ penalty='elasticnet'
        "clf__C": [1e-5, 3e-5, 1e-4, 3e-4, 1e-3, 3e-3, 1e-2, 0.05, 0.1, 0.2, 0.5,
                   1, 2, 5, 10, 20, 50, 100, 200, 500, 1_000, 2_000, 5_000],
        "clf__tol": [1e-2, 3e-3, 1e-3, 3e-4, 1e-4, 3e-5, 1e-5, 1e-6],
        "clf__max_iter": [200, 500, 1_000, 2_000, 5_000, 10_000],
        "clf__fit_intercept": [True, False],
        "clf__class_weight": [None, "balanced"],
        "clf__multi_class": ["auto", "ovr", "multinomial"],
        "clf__warm_start": [False, True],
    },
]

best_lr = run_grid_search(lr_pipe, grid_lr_long)
joblib.dump(best_lr, 'models/best_lr_model.joblib')

### - Decision Tree

In [ ]:
dt_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", DecisionTreeClassifier(random_state=42))
])

grid_dt = {
    "clf__max_depth": [None, 5, 10, 15],  # กัน overfit
    "clf__min_samples_leaf": [1, 3, 5, 10],
    "clf__class_weight": [None, "balanced"]
}

best_dt = run_grid_search(dt_pipe, grid_dt)


### - Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", RandomForestClassifier(random_state=42, n_jobs=-1))
])

grid_rf = {
    "clf__n_estimators": [200, 400, 600],  # มากขึ้น = เสถียรขึ้น (แต่ช้าขึ้น)
    "clf__max_depth": [None, 10, 15, 20],
    "clf__min_samples_leaf": [1, 2, 4],
    "clf__max_features": ["sqrt", "log2"], # 'sqrt' มักดีสำหรับ classification
    "clf__class_weight": [None, "balanced"]
}

best_rf = run_grid_search(rf_pipe, grid_rf)


In [ ]:
# rf_pipe = Pipeline([
#     ("scaler", StandardScaler()),
#     ("clf", RandomForestClassifier(random_state=42, n_jobs=-1))
# ])
#
# grid_rf = {
#     # จำนวนต้นไม้ (เยอะขึ้น เสถียรขึ้น แต่ช้าลง)
#     "clf__n_estimators": [200, 400, 600, 1000],
#
#     # ความลึกของต้นไม้
#     "clf__max_depth": [None, 10, 15, 25, 30],
#
#     # จำนวน sample ขั้นต่ำต่อ split และ leaf
#     "clf__min_samples_split": [2, 5, 10],
#     "clf__min_samples_leaf": [1, 2, 6],
#
#     # ฟีเจอร์ที่ใช้ตอน split
#     "clf__max_features": ["sqrt", "log2", None],
#
#     # ถ่วงน้ำหนักคลาส
#     "clf__class_weight": [None, "balanced"]
# }
#
# best_rf = run_grid_search(rf_pipe, grid_rf)
#
# # Export the trained RandomForest model
# joblib.dump(best_rf, 'models/best_rf_model.joblib')
# print('Model exported to models/best_rf_model.joblib')


In [ ]:
from sklearn.decomposition import PCA

# Pipeline: เพิ่ม PCA คั่นกลางเพื่อเพิ่มมิติการค้นหา
lr_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("pca", PCA(random_state=0)),             # ใส่ PCA เสมอ แต่กำหนด n_components ผ่านกริด
    ("clf", LogisticRegression(solver="lbfgs"))
])

# กริดขยายใหญ่ (~13,824 combinations)
grid_lr = {
    # ----- PCA (6 คอมโบ) -----
    # None = ไม่ลดมิติ (ยังคงคำนวณ PCA อยู่ → ช่วยเพิ่มเวลา), 0.95/0.99 = คงความแปรปรวนตามสัดส่วน
    "pca__n_components": [None, 0.95, 0.99],
    "pca__whiten": [True, False],             # ทำให้คอมโบเพิ่มขึ้นและช้าขึ้นเล็กน้อย
    # (คง svd_solver เป็น "auto" เพื่อเลี่ยงคอมโบเสี่ยง)

    # ----- LogisticRegression (2,304 คอมโบ) -----
    # C แบบละเอียด (log-space) ครอบคลุมกว้างมาก
    "clf__C": list(np.logspace(-4, 3, 24)),   # 24 ค่า ตั้งแต่ 1e-4 ถึง 1e3
    # ลด tol ให้เล็กลงเพื่อยืดเวลา iterate
    "clf__tol": [1e-3, 1e-4, 1e-5, 1e-6],     # 4 ค่า
    # เพิ่มรอบสูง เพื่อกันหลุดหยุดเร็ว
    "clf__max_iter": [5000, 10000, 20000],    # 3 ค่า
    # ถ่วง class
    "clf__class_weight": [None, "balanced"],  # 2 ค่า
    # ลองมี/ไม่มี intercept
    "clf__fit_intercept": [True, False],      # 2 ค่า
    # warm_start มี/ไม่มี (แม้ grid แต่ละตัวเป็น estimator ใหม่ ก็ยังเพิ่มเวลา/คอมโบได้)
    "clf__warm_start": [True, False],         # 2 ค่า
}

best_lr = run_grid_search(lr_pipe, grid_lr)

In [ ]:
# Pipeline ที่ fix ค่า hyperparameters ตาม best params
rf_best = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", RandomForestClassifier(
        random_state=42,
        n_jobs=-1,
        class_weight=None,
        max_depth=10,
        max_features="sqrt",
        min_samples_leaf=6,
        min_samples_split=2,
        n_estimators=600
    ))
])

# เทรนโมเดล
rf_best.fit(X_train, y_train)

# ถ้าอยากดูผล
y_pred = rf_best.predict(X_test)

print("Accuracy:", (y_pred == y_test).mean())

joblib.dump(rf_best, 'models/best_rf_model.joblib')